# 4.2 データ整形

In [1]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [2]:
# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)
# データフレームでの表示精度
pd.options.display.float_format = '{:.4f}'.format
# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# 乱数の種
random_seed = 123

In [4]:
# 追加ライブラリのimport
import seaborn as sns

# サンプルデータの読み込み
df_titanic = sns.load_dataset("titanic")

# 項目名の日本語化
columns_t = ['生存', '等室', '性別', '年齢', '兄弟配偶者数', 
             '両親子供数', '料金', '乗船港コード', '等室名', 
             '男女子供', '成人男子', 'デッキ', '乗船港', '生存可否', '独身']
df_titanic.columns = columns_t

### データ確認

In [5]:
display(df_titanic.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港コード,等室名,男女子供,成人男子,デッキ,乗船港,生存可否,独身
0,0,3,male,22.0000,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0000,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0000,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0000,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0000,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [6]:
print(df_titanic.shape)

(891, 15)


### 4.2.1 不要列削除

In [7]:
# 余分な列削除

# 「等室名」 (「等室」と同じ)
df1 = df_titanic.drop('等室名', axis=1)

# 「乗船港」 (「乗船港コード」と同じ)
df2 = df1.drop('乗船港', axis=1)

# 「生存可否」 (「生存」と同じ)
df3 = df2.drop('生存可否', axis=1)

# 結果確認
display(df3.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港コード,男女子供,成人男子,デッキ,独身
0,0,3,male,22.0000,1,0,7.2500,S,man,True,NaN,False
1,1,1,female,38.0000,1,0,71.2833,C,woman,False,C,False
2,1,3,female,26.0000,0,0,7.9250,S,woman,False,NaN,True
3,1,1,female,35.0000,1,0,53.1000,S,woman,False,C,False
4,0,3,male,35.0000,0,0,8.0500,S,man,True,NaN,True


### 4.2.2 欠損値対応

In [8]:
# 欠損値確認
display(df3.isnull().sum())

生存          0
等室          0
性別          0
年齢        177
兄弟配偶者数      0
両親子供数       0
料金          0
乗船港コード      2
男女子供        0
成人男子        0
デッキ       688
独身          0
dtype: int64

In [9]:
display(df3['デッキ'].value_counts())

C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: デッキ, dtype: int64

#### 方針の決定

**乗船港コード**: 件数が2件と少ない
-> 行ごと削除する      

**年齢**: 数値データであり、欠損行数が177件とかなり多い
-> 他データの平均値で代用  

**デッキ**: コード値データであり、欠損行数が688行と相当多い
-> 欠損を意味するダミーコードを振って全行処理対象とする

In [10]:
# 乗船港コード: 件数が2件と少ない 
# -> 行ごと削除する

# dropna関数を利用する
df4 = df3.dropna(subset = ['乗船港コード'])

# 年齢: 数値データであり欠損行数が177件とかなり多い 
# -> 他データの平均値で代用

# 平均値の計算
age_average = df4['年齢'].mean()

# fillna関数の利用
df5 = df4.fillna({'年齢': age_average})

# デッキ: コード値データであり欠損行数が688行と相当多い
# -> 欠損を意味するダミーコードを振って全行処理対象とする

# replace関数の利用  (ダミーコードは 'N' とする)
df6 = df5.replace({'デッキ': {np.nan: 'N'}})

In [11]:
# 結果確認
display(df6.isnull().sum())

display(df6.head())

生存        0
等室        0
性別        0
年齢        0
兄弟配偶者数    0
両親子供数     0
料金        0
乗船港コード    0
男女子供      0
成人男子      0
デッキ       0
独身        0
dtype: int64

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港コード,男女子供,成人男子,デッキ,独身
0,0,3,male,22.0000,1,0,7.2500,S,man,True,N,False
1,1,1,female,38.0000,1,0,71.2833,C,woman,False,C,False
2,1,3,female,26.0000,0,0,7.9250,S,woman,False,N,True
3,1,1,female,35.0000,1,0,53.1000,S,woman,False,C,False
4,0,3,male,35.0000,0,0,8.0500,S,man,True,N,True


### 4.2.3 二値ラベルの数値化

**性別**: male / female  
**成人男子**: True / False  
**独身**: True / False  
    
をそれぞれ 1/ 0 に数値化する


#### 「性別」の数値化

In [12]:
display(df6['性別'].value_counts())

male      577
female    312
Name: 性別, dtype: int64

In [13]:
# 辞書 mf_map の定義
mf_map = {'male': 1, 'female': 0}

# map関数を利用して数値化
df7 = df6.copy()
df7['性別'] = df7['性別'].map(mf_map)

# 結果確認
display(df7.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港コード,男女子供,成人男子,デッキ,独身
0,0,3,1,22.0000,1,0,7.2500,S,man,True,N,False
1,1,1,0,38.0000,1,0,71.2833,C,woman,False,C,False
2,1,3,0,26.0000,0,0,7.9250,S,woman,False,N,True
3,1,1,0,35.0000,1,0,53.1000,S,woman,False,C,False
4,0,3,1,35.0000,0,0,8.0500,S,man,True,N,True


#### 「成人男子」「独身」の数値化

In [14]:
display(df7['成人男子'].value_counts())

True     537
False    352
Name: 成人男子, dtype: int64

In [15]:
# 辞書 tf_map の定義
tf_map = {True: 1, False: 0}

# map関数を利用して数値化
df8 = df7.copy()
df8['成人男子'] = df8['成人男子'].map(tf_map)

# map関数を利用して数値化
df9 = df8.copy()
df9['独身'] = df8['独身'].map(tf_map)

# 結果確認
display(df9.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港コード,男女子供,成人男子,デッキ,独身
0,0,3,1,22.0000,1,0,7.2500,S,man,1,N,0
1,1,1,0,38.0000,1,0,71.2833,C,woman,0,C,0
2,1,3,0,26.0000,0,0,7.9250,S,woman,0,N,1
3,1,1,0,35.0000,1,0,53.1000,S,woman,0,C,0
4,0,3,1,35.0000,0,0,8.0500,S,man,1,N,1


### 4.2.4 多値ラベルの数値化 
(One Hot Vector化)

In [16]:
# 変換前
display(df9[['男女子供']].head(10))

,男女子供
0,man
1,woman
2,woman
3,woman
4,man
5,man
6,man
7,child
8,woman
9,child


In [17]:
# get_dummies関数の利用サンプル

w = pd.get_dummies(df9['男女子供'], prefix='男女子供')
display(w.head(10))

,男女子供_child,男女子供_man,男女子供_woman
0,0,1,0
1,0,0,1
2,0,0,1
3,0,0,1
4,0,1,0
5,0,1,0
6,0,1,0
7,1,0,0
8,0,0,1
9,1,0,0


In [18]:
# get_dummies関数でカテゴリ値をone hot vectorに展開する関数を定義
# df 対象データフレーム
# column 対象列

def enc(df, column):
    # One Hot Vector生成
    df_dummy = pd.get_dummies(df[column], prefix=column)
    # 元列の削除
    df_drop = df.drop([column], axis=1)
    # 削除したデータフレームと、One Hot生成列を連結
    df1 = pd.concat([df_drop,df_dummy],axis=1)
    return df1

#### 男女子供
man / woman / child

In [19]:
# 項目値の確認
display(df9['男女子供'].value_counts())

man      537
woman    269
child     83
Name: 男女子供, dtype: int64

In [20]:
# One-Hot エンコード化

# 男女子供,
df10 = enc(df9, '男女子供')

# 結果確認
display(df10.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港コード,成人男子,デッキ,独身,男女子供_child,男女子供_man,男女子供_woman
0,0,3,1,22.0000,1,0,7.2500,S,1,N,0,0,1,0
1,1,1,0,38.0000,1,0,71.2833,C,0,C,0,0,0,1
2,1,3,0,26.0000,0,0,7.9250,S,0,N,1,0,0,1
3,1,1,0,35.0000,1,0,53.1000,S,0,C,0,0,0,1
4,0,3,1,35.0000,0,0,8.0500,S,1,N,1,0,1,0


#### 乗船港コードとデッキ

In [21]:
# One-Hot エンコード化

# 乗船港コード
df11 = enc(df10, '乗船港コード')

# デッキ
df12 = enc(df11, 'デッキ')

# 結果確認
display(df12.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,成人男子,独身,男女子供_child,男女子供_man,男女子供_woman,乗船港コード_C,乗船港コード_Q,乗船港コード_S,デッキ_A,デッキ_B,デッキ_C,デッキ_D,デッキ_E,デッキ_F,デッキ_G,デッキ_N
0,0,3,1,22.0000,1,0,7.2500,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1
1,1,1,0,38.0000,1,0,71.2833,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0
2,1,3,0,26.0000,0,0,7.9250,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1
3,1,1,0,35.0000,1,0,53.1000,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0
4,0,3,1,35.0000,0,0,8.0500,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1


### 4.2.5 正規化

In [22]:
# standardization 

df13 = df12.copy()
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
df13[['年齢', '料金']] = stdsc.fit_transform(df13[['年齢', '料金']])

# 結果確認
display(df13.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,成人男子,独身,男女子供_child,男女子供_man,男女子供_woman,乗船港コード_C,乗船港コード_Q,乗船港コード_S,デッキ_A,デッキ_B,デッキ_C,デッキ_D,デッキ_E,デッキ_F,デッキ_G,デッキ_N
0,0,3,1,-0.5896,1,0,-0.5002,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1
1,1,1,0,0.6448,1,0,0.7889,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0
2,1,3,0,-0.2810,0,0,-0.4866,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1
3,1,1,0,0.4134,1,0,0.4229,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0
4,0,3,1,0.4134,0,0,-0.4841,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1


### 4.2.6 離散化

In [23]:
# 離散化の例

# 境界値のリスト
bins = [0, 5, 10, 20, 30, 40, 50, 60, 70, 80]

# 離散化の実施
df14 = df12.copy()
df14['年齢_bin'] = pd.cut(df14['年齢'], bins, labels=False)

# 結果確認
display(df14[['年齢', '年齢_bin']].head(10))

,年齢,年齢_bin
0,22.0000,3
1,38.0000,4
2,26.0000,3
3,35.0000,4
4,35.0000,4
5,29.6421,3
6,54.0000,6
7,2.0000,0
8,27.0000,3
9,14.0000,2


##  以下はおまけ。試しにモデルを作ってみた

In [ ]:
x = df12.drop(['生存'], axis=1)
y = df12['生存']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(\
  x, y, train_size=0.7, test_size=0.3, random_state=random_seed)

In [ ]:
x2 = df13.drop(['生存'], axis=1)
y2 = df13['生存']

from sklearn.model_selection import train_test_split
x2_train, x2_test, y2_train, y2_test = train_test_split(\
  x2, y2, train_size=0.7, test_size=0.3, random_state=random_seed)

In [ ]:
import xgboost
bst = xgboost.XGBClassifier()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
bst.fit(x_train, y_train)
print(bst.score(x_test, y_test))
y_pred = bst.predict(x_test)
matrix = confusion_matrix(y_test.values, y_pred)
df_matrix = pd.DataFrame(matrix, columns=['0', '1'], index=['0', '1'])
display(df_matrix)
print(classification_report(y_test, y_pred))

In [ ]:
bst2 = xgboost.XGBClassifier()
bst2.fit(x2_train, y2_train)
print(bst2.score(x2_test, y2_test))
y2_pred = bst2.predict(x2_test)
matrix2 = confusion_matrix(y2_test.values, y2_pred)
df_matrix2 = pd.DataFrame(matrix2, columns=['0', '1'], index=['0', '1'])
display(df_matrix2)
print(classification_report(y2_test, y2_pred))